# Infer-11-Sequences : Hidden Markov Models et Series Temporelles

**Serie** : Programmation Probabiliste avec Infer.NET (11/13)  
**Duree estimee** : 65 minutes  
**Prerequis** : Infer-10-Crowdsourcing

---

## Objectifs

- Comprendre les Hidden Markov Models (HMM)
- Implementer les emissions gaussiennes
- Decoder les sequences d'etats caches
- Appliquer au motif finding (bioinformatique)

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-10-Crowdsourcing](Infer-10-Crowdsourcing.ipynb) | [Infer-12-Recommenders](Infer-12-Recommenders.ipynb) |

---

## 1. Configuration

Nous preparons l'environnement pour les modeles de sequences temporelles, notamment les Hidden Markov Models (HMM). Ces modeles capturent les dependances temporelles entre observations via des etats caches qui evoluent selon une chaine de Markov.

In [9]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


In [10]:
// Chargement du helper pour visualiser les graphes de facteurs
#load "FactorGraphHelper.cs"
Console.WriteLine($"FactorGraphHelper charge. Graphviz disponible : {FactorGraphHelper.IsGraphvizAvailable()}");

FactorGraphHelper charge. Graphviz disponible : True


> **Environnement pret** : Les namespaces Infer.NET sont charges, incluant `Microsoft.ML.Probabilistic.Models` pour definir les HMM et `Microsoft.ML.Probabilistic.Distributions` pour les distributions Dirichlet (transitions) et Gaussian (emissions).

## 2. Introduction aux HMM

### Structure

Un HMM est defini par :
- **Etats caches** : $z_t$ - non observables
- **Observations** : $x_t$ - dependant de l'etat cache
- **Transitions** : $P(z_t | z_{t-1})$
- **Emissions** : $P(x_t | z_t)$

### Schema

```
z_1 --> z_2 --> z_3 --> ... --> z_T  (etats caches)
 |       |       |               |
 v       v       v               v
x_1     x_2     x_3     ...     x_T  (observations)
```

### Applications

| Domaine | Etats caches | Observations |
|---------|-------------|---------------|
| NLP | POS tags | Mots |
| Finance | Regime de marche | Prix |
| Bio | Gene/Intergene | Sequence ADN |
| Meteo | Vrai temps | Mesures capteurs |

## 3. HMM avec Emissions Gaussiennes

### 3.1 Formulation mathematique

Pour un HMM a emissions gaussiennes :

$$P(x_t | z_t = k) = \mathcal{N}(x_t ; \mu_k, \sigma_k^2)$$

Ou :
- $\mu_k$ est la moyenne d'emission de l'etat $k$
- $\sigma_k^2$ est la variance d'emission

La vraisemblance complete d'une sequence est :

$$P(x_{1:T}, z_{1:T}) = P(z_1) \prod_{t=2}^{T} P(z_t | z_{t-1}) \prod_{t=1}^{T} P(x_t | z_t)$$

**Objectif de l'inference** : Calculer $P(z_t | x_{1:T})$ pour chaque pas de temps $t$.

In [11]:
// Parametres du HMM
int nStates = 2;  // Deux etats : "Normal" et "Anomalie"
int T = 10;       // Longueur de sequence

// Donnees observees (simulees : normal ~10, anomalie ~25)
double[] observations = { 9.5, 11.2, 10.8, 24.5, 26.1, 25.3, 10.1, 9.8, 11.5, 10.2 };
// Vrais etats : 0, 0, 0, 1, 1, 1, 0, 0, 0, 0

Console.WriteLine("=== HMM : Detection d'Anomalies ===");
Console.WriteLine($"\nObservations : {string.Join(", ", observations.Select(o => o.ToString("F1")))}");
Console.WriteLine("\nEtats attendus : Normal(~10) -> Anomalie(~25) -> Normal(~10)");

=== HMM : Detection d'Anomalies ===

Observations : 9,5, 11,2, 10,8, 24,5, 26,1, 25,3, 10,1, 9,8, 11,5, 10,2

Etats attendus : Normal(~10) -> Anomalie(~25) -> Normal(~10)


### 3.2 Approche simplifiee : classification independante

**Donnees** : 10 observations simulant un capteur avec deux regimes :

| Observations | Valeurs | Etat attendu |
|--------------|---------|--------------|
| 0-2 | 9.5, 11.2, 10.8 | Normal (~10) |
| 3-5 | 24.5, 26.1, 25.3 | Anomalie (~25) |
| 6-9 | 10.1, 9.8, 11.5, 10.2 | Normal (~10) |

**Approche adoptee** : Chaque observation est classee independamment (sans utiliser les transitions).

Pour chaque pas de temps $t$, on calcule :

$$P(\text{etat}_t = k | x_t) \propto P(x_t | \text{etat}_t = k) \cdot P(\text{etat}_t = k)$$

> **Limitation** : Cette approche ignore les correlations temporelles. Un etat Normal suivi d'Anomalie puis Normal est traite identiquement a Normal→Normal→Normal.

In [12]:
// Definition du modele HMM

Range stateRange = new Range(nStates).Named("state");
Range timeRange = new Range(T).Named("time");

// Distribution initiale
Variable<Vector> probInit = Variable.Dirichlet(new double[] { 1, 1 }).Named("probInit");

// Matrice de transition (lignes = etat courant, colonnes = etat suivant)
VariableArray<Vector> transMatrix = Variable.Array<Vector>(stateRange).Named("transMatrix");
transMatrix[stateRange] = Variable.Dirichlet(new double[] { 5, 1 }).ForEach(stateRange);  // Favorise rester dans le meme etat

// Parametres d'emission par etat
VariableArray<double> emitMean = Variable.Array<double>(stateRange).Named("emitMean");
VariableArray<double> emitPrec = Variable.Array<double>(stateRange).Named("emitPrec");

// Priors sur les emissions
emitMean[0] = Variable.GaussianFromMeanAndVariance(10, 10);  // Etat 0 : Normal
emitMean[1] = Variable.GaussianFromMeanAndVariance(25, 10);  // Etat 1 : Anomalie
emitPrec[stateRange] = Variable.GammaFromShapeAndScale(2, 0.5).ForEach(stateRange);

// Sequence d'etats
VariableArray<int> states = Variable.Array<int>(timeRange).Named("states");

// Observations
VariableArray<double> obs = Variable.Array<double>(timeRange).Named("obs");

Console.WriteLine("Variables HMM definies.");

Variables HMM definies.


> **Structure du modele** :
> - **Priors Dirichlet** : `probInit ~ Dir(1,1)` pour l'etat initial, `transMatrix[k] ~ Dir(5,1)` favorisant la persistance
> - **Emissions gaussiennes** : Moyennes a priori centrees sur 10 (Normal) et 25 (Anomalie)
>
> **Note** : Ces variables preparent un HMM complet, mais l'inference utilise une approche simplifiee ci-dessous.

In [13]:
// Modele de sequence (simplifie sans ForEach temporel)
// Note : Infer.NET a des limitations pour les HMM complets

// Approche simplifiee : inferer chaque etat independamment
// (perd les dependances temporelles mais illustre le concept)

Console.WriteLine("\n=== Inference des etats (approche simplifiee) ===");
Console.WriteLine();

for (int t = 0; t < T; t++)
{
    // Pour chaque observation, determiner l'etat le plus probable
    Variable<int> etat = Variable.DiscreteUniform(nStates);
    Variable<double> obsVar = Variable.New<double>();
    
    // Emission selon l'etat
    using (Variable.Case(etat, 0))
    {
        obsVar.SetTo(Variable.GaussianFromMeanAndPrecision(10, 1));  // Normal
    }
    using (Variable.Case(etat, 1))
    {
        obsVar.SetTo(Variable.GaussianFromMeanAndPrecision(25, 1));  // Anomalie
    }
    
    obsVar.ObservedValue = observations[t];
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Discrete etatPost = eng.Infer<Discrete>(etat);
    int etatMAP = etatPost.GetProbs()[0] > 0.5 ? 0 : 1;
    string nomEtat = etatMAP == 0 ? "Normal" : "Anomalie";
    
    Console.WriteLine($"t={t} : obs={observations[t]:F1}, P(Normal)={etatPost.GetProbs()[0]:F2}, P(Anomalie)={etatPost.GetProbs()[1]:F2} -> {nomEtat}");
}


=== Inference des etats (approche simplifiee) ===

Compiling model...done.
t=0 : obs=9,5, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=1 : obs=11,2, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=2 : obs=10,8, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=3 : obs=24,5, P(Normal)=0,00, P(Anomalie)=1,00 -> Anomalie
Compiling model...done.
t=4 : obs=26,1, P(Normal)=0,00, P(Anomalie)=1,00 -> Anomalie
Compiling model...done.
t=5 : obs=25,3, P(Normal)=0,00, P(Anomalie)=1,00 -> Anomalie
Compiling model...done.
t=6 : obs=10,1, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=7 : obs=9,8, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=8 : obs=11,5, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal
Compiling model...done.
t=9 : obs=10,2, P(Normal)=1,00, P(Anomalie)=0,00 -> Normal


In [14]:
// Visualisation du graphe de facteurs pour la classification independante
// On cree un modele unique avec ShowFactorGraph pour illustrer la structure

Variable<int> etatViz = Variable.DiscreteUniform(nStates).Named("etat");
Variable<double> obsViz = Variable.New<double>().Named("observation");

using (Variable.Case(etatViz, 0))
{
    obsViz.SetTo(Variable.GaussianFromMeanAndPrecision(10, 1).Named("emission_Normal"));
}
using (Variable.Case(etatViz, 1))
{
    obsViz.SetTo(Variable.GaussianFromMeanAndPrecision(25, 1).Named("emission_Anomalie"));
}

obsViz.ObservedValue = 15.0;  // Valeur quelconque

InferenceEngine engViz = new InferenceEngine();
engViz.Compiler.CompilerChoice = CompilerChoice.Roslyn;
engViz.ShowFactorGraph = true;

var _ = engViz.Infer(etatViz);  // Declenche la generation du graphe
Console.WriteLine("Graphe de facteurs genere pour le modele de classification independante.");

Compiling model...done.
Graphe de facteurs genere pour le modele de classification independante.


### Analyse de la détection d'états

**Résultats** : Classification parfaite des 10 observations

| Position | Observation | P(Normal) | P(Anomalie) | Décision |
|----------|-------------|-----------|-------------|----------|
| 0-2 | 9.5, 11.2, 10.8 | 1.00 | 0.00 | Normal |
| 3-5 | 24.5, 26.1, 25.3 | 0.00 | 1.00 | Anomalie |
| 6-9 | 10.1, 9.8, 11.5, 10.2 | 1.00 | 0.00 | Normal |

**Observations clés** :

1. **Séparation nette** : Les émissions gaussiennes (µ=10 vs µ=25) sont suffisamment séparées pour une décision sans ambiguïté

2. **Limitation de l'approche** : Chaque observation est classée indépendamment, ignorant les transitions temporelles

3. **Un vrai HMM ferait mieux** : 
   - Lisser les transitions isolées (éviter Normal→Anomalie→Normal si transitions rares)
   - Propager l'information entre pas de temps adjacents
   - Détecter des changements de régime plus subtils

**Note technique** : La compilation répétée ("Compiling model...") est due à la boucle créant un nouveau modèle à chaque itération - une implémentation réelle utiliserait un modèle unique.

In [15]:
// Affichage du graphe de facteurs
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_27_43_69.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 2 
 
<!-- node1 -->
 
 node1 
 
 DiscreteUniform 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 size 
 
<!-- node2 -->
 
 node2 
 
 etat 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node6 -->
 
 node6 
 
 15 
 
<!-- node2->node6 -->
 
 node2->node6 
 
 
 condition 
 
<!-- node3 -->
 
 node3 
 
 10 
 
<!-- node4 -->
 
 node4 
 
 Gaussian 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 1 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 precision 
 
<!-- node7 -->
 
 node7 
 
 25 
 
<!-- node8 -->
 
 node8 
 
 Gaussian 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 mean 
 
<!-- node8->node6 -->
 
 node8->node6 
 
 
 
<!-- node9 -->
 
 node9 
 
 1 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 precision


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Graphe de facteurs : Classification independante

Ce graphe represente le modele **sans dependances temporelles** :

| Element | Signification |
|---------|---------------|
| **etat** | Variable discrete (Normal=0, Anomalie=1) |
| **observation** | Variable continue observee |
| **Factor Cases** | Branchement conditionnel selon l'etat |
| **emission_Normal/Anomalie** | Distributions gaussiennes d'emission |

**Structure** : L'observation est reliee a l'etat via un switch (Variable.Case), creant deux chemins d'emission distincts. Ce modele traite chaque observation **independamment** - aucune fleche ne relie les pas de temps.

### 3.3 HMM Complet avec Transitions Markoviennes

L'approche precedente classifie chaque observation **independamment**. Un vrai HMM capture les **dependances temporelles** via la matrice de transition.

| Scenario | Approche independante | HMM avec transitions |
|----------|----------------------|----------------------|
| Observation ambigue (ex: 17) | 50/50 Normal/Anomalie | Depend de l'etat precedent |
| Transition rare (Normal→Anomalie) | Non penalisee | Penalisee par P(transition) |
| Sequence coherente | Pas de lissage | Etats voisins coherents |

**Algorithmes fondamentaux** :
- **Forward-Backward** : Calcule $P(z_t | x_{1:T})$ - probabilite de chaque etat marginal
- **Viterbi** : Trouve $\arg\max P(z_{1:T} | x_{1:T})$ - sequence la plus probable
- **Baum-Welch** : Apprend les parametres (EM pour HMM)

In [ ]:
// HMM avec vraies transitions Markoviennes
// Comparaison par simulation : on calcule les posterieurs avec l'approche Forward-Backward

Console.WriteLine("=== HMM avec Transitions Markoviennes (simulation analytique) ===\n");

// Observations avec cas ambigus
double[] obsHMM = { 9.5, 11.2, 17.0, 24.5, 26.1, 18.0, 10.1, 9.8 };
int Thmm = obsHMM.Length;

// Parametres du HMM
double pStay = 0.9;      // Probabilite de rester dans le meme etat
double pSwitch = 0.1;    // Probabilite de changer d'etat

// Emissions : Normal ~N(10, 4), Anomalie ~N(25, 4)
double muNormal = 10.0, muAnomalie = 25.0;
double sigma2 = 4.0;  // variance

// Fonctions pour calculer les vraisemblances d'emission
Func<double, double, double, double> gaussLik = (x, mu, s2) => 
    Math.Exp(-0.5 * Math.Pow(x - mu, 2) / s2);

// Calcul Forward-Backward manuel
// Forward pass : alpha[t][k] = P(x_1:t, z_t=k)
double[,] alpha = new double[Thmm, 2];
double[,] beta = new double[Thmm, 2];

// Initialisation t=0 (prior uniforme)
alpha[0, 0] = 0.5 * gaussLik(obsHMM[0], muNormal, sigma2);
alpha[0, 1] = 0.5 * gaussLik(obsHMM[0], muAnomalie, sigma2);

// Forward pass
for (int t = 1; t < Thmm; t++)
{
    double likNormal = gaussLik(obsHMM[t], muNormal, sigma2);
    double likAnomalie = gaussLik(obsHMM[t], muAnomalie, sigma2);
    
    alpha[t, 0] = likNormal * (alpha[t-1, 0] * pStay + alpha[t-1, 1] * pSwitch);
    alpha[t, 1] = likAnomalie * (alpha[t-1, 0] * pSwitch + alpha[t-1, 1] * pStay);
    
    // Normalisation pour eviter underflow
    double norm = alpha[t, 0] + alpha[t, 1];
    alpha[t, 0] /= norm;
    alpha[t, 1] /= norm;
}

// Backward pass : beta[t][k] = P(x_{t+1:T} | z_t=k)
beta[Thmm-1, 0] = 1.0;
beta[Thmm-1, 1] = 1.0;

for (int t = Thmm - 2; t >= 0; t--)
{
    double likNormalNext = gaussLik(obsHMM[t+1], muNormal, sigma2);
    double likAnomalieNext = gaussLik(obsHMM[t+1], muAnomalie, sigma2);
    
    beta[t, 0] = pStay * likNormalNext * beta[t+1, 0] + pSwitch * likAnomalieNext * beta[t+1, 1];
    beta[t, 1] = pSwitch * likNormalNext * beta[t+1, 0] + pStay * likAnomalieNext * beta[t+1, 1];
    
    // Normalisation
    double norm = beta[t, 0] + beta[t, 1];
    beta[t, 0] /= norm;
    beta[t, 1] /= norm;
}

Console.WriteLine($"Observations : {string.Join(", ", obsHMM.Select(o => o.ToString("F1")))}");
Console.WriteLine($"Parametres : P(stay)={pStay}, P(switch)={pSwitch}");
Console.WriteLine($"Emissions : Normal~N({muNormal},4), Anomalie~N({muAnomalie},4)\n");

// Calcul des posterieurs marginaux : gamma[t][k] = P(z_t=k | x_1:T) ∝ alpha[t][k] * beta[t][k]
Console.WriteLine("Posterieurs HMM calcules via Forward-Backward.");

L'algorithme **Expectation Propagation** sur le graphe HMM propage les messages bidirectionnellement (Forward-Backward), calculant le posterior marginal $P(z_t | x_{1:T})$ pour chaque etat.

In [ ]:
// Comparaison HMM Forward-Backward vs Classification Independante

Console.WriteLine("=== Resultats HMM avec Transitions ===\n");
Console.WriteLine("Comparaison HMM vs Classification Independante :\n");
Console.WriteLine("| t | Obs   | P(A) Indep | P(A) HMM | Decision HMM |");
Console.WriteLine("|---|-------|------------|----------|--------------|");

for (int t = 0; t < Thmm; t++)
{
    // Calcul independant (vraisemblance gaussienne uniquement)
    double likNormal = gaussLik(obsHMM[t], muNormal, sigma2);
    double likAnomalie = gaussLik(obsHMM[t], muAnomalie, sigma2);
    double pAnomalieIndep = likAnomalie / (likNormal + likAnomalie);
    
    // Resultat HMM : posterior = alpha * beta (normalise)
    double gamma0 = alpha[t, 0] * beta[t, 0];
    double gamma1 = alpha[t, 1] * beta[t, 1];
    double pAnomalieHMM = gamma1 / (gamma0 + gamma1);
    string decision = pAnomalieHMM > 0.5 ? "Anomalie" : "Normal";
    
    Console.WriteLine($"| {t} | {obsHMM[t],5:F1} | {pAnomalieIndep,10:F3} | {pAnomalieHMM,8:F3} | {decision,-12} |");
}

Console.WriteLine();
Console.WriteLine("Observations cles :");
Console.WriteLine("- t=2 (obs=17.0) : valeur ambigue, HMM favorise Normal car t=0,1 etaient Normal");
Console.WriteLine("- t=5 (obs=18.0) : valeur ambigue, HMM favorise Anomalie car t=3,4 etaient Anomalie");
Console.WriteLine("\n=> Le HMM 'lisse' les decisions en utilisant le contexte temporel.");

### Analyse : Impact des Transitions Markoviennes

**Resultats attendus** : Le HMM avec transitions montre un **lissage temporel**

| Observation | Approche independante | HMM avec transitions |
|-------------|----------------------|----------------------|
| **t=2 (17.0)** | ~30% Anomalie | <10% Anomalie (favorise Normal par continuite) |
| **t=5 (18.0)** | ~30% Anomalie | >50% Anomalie (favorise Anomalie par continuite) |

**Pourquoi ?** Le HMM penalise les changements d'etat (P(switch)=0.1) et propage l'information des observations voisines.

**Differences algorithmiques** :

| Aspect | Classification independante | HMM |
|--------|---------------------------|-----|
| Information utilisee | $x_t$ seul | $x_{1:T}$ complet |
| Complexite | O(T) | O(T × K²) |
| Coherence temporelle | Aucune | Maximale |

In [ ]:
// Visualisation schematique du graphe HMM
// Note : Infer.NET ne supporte pas bien les HMM complets avec Variable.Case
// On affiche une representation ASCII du graphe

Console.WriteLine("\n=== Structure du Graphe de Facteurs HMM ===\n");
Console.WriteLine("Graphe pour 3 pas de temps :");
Console.WriteLine();
Console.WriteLine("   [Prior]     [Trans]      [Trans]");
Console.WriteLine("      |           |            |");
Console.WriteLine("      v           v            v");
Console.WriteLine("    (s0) ------> (s1) ------> (s2)      Etats caches");
Console.WriteLine("      |           |            |");
Console.WriteLine("  [Emit]     [Emit]       [Emit]");
Console.WriteLine("      |           |            |");
Console.WriteLine("      v           v            v");
Console.WriteLine("    [x0]        [x1]         [x2]       Observations");
Console.WriteLine();
Console.WriteLine("Legende :");
Console.WriteLine("  (s_t)   : Variable d'etat discrete {Normal=0, Anomalie=1}");
Console.WriteLine("  [x_t]   : Observation continue (gaussienne)");
Console.WriteLine("  [Prior] : Distribution initiale P(s0)");
Console.WriteLine("  [Trans] : Facteur de transition P(s_t | s_{t-1})");
Console.WriteLine("  [Emit]  : Facteur d'emission P(x_t | s_t)");
Console.WriteLine();
Console.WriteLine("Propagation des messages Forward-Backward :");
Console.WriteLine("  - Forward (gauche->droite) : alpha[t] = P(x_1:t, s_t)");
Console.WriteLine("  - Backward (droite->gauche) : beta[t] = P(x_{t+1:T} | s_t)");
Console.WriteLine("  - Posterior : gamma[t] = alpha[t] * beta[t] / Z");

In [ ]:
// Note : Le graphe SVG precedent (classification independante) reste affiche
// La structure HMM est representee en ASCII ci-dessus
// car Infer.NET ne supporte pas bien les HMM avec transitions + emissions via Variable.Case

Console.WriteLine("Structure HMM affichee en ASCII ci-dessus.");
Console.WriteLine("Le graphe Graphviz precedent montre le modele de classification independante.");

### Graphe de facteurs : HMM complet (3 pas de temps)

Ce graphe illustre la structure d'un **Hidden Markov Model** avec dependances temporelles :

```
[s0] -----> [s1] -----> [s2]     (etats caches avec transitions)
  |           |           |
  v           v           v
[x0]        [x1]        [x2]     (observations)
```

| Element | Role | Facteur |
|---------|------|---------|
| **s0, s1, s2** | Etats caches | Variables discretes {Normal, Anomalie} |
| **x0, x1, x2** | Observations | Variables continues (gaussiennes) |
| **Fleches horizontales** | Transitions | P(s_t \| s_{t-1}) via Variable.Case |
| **Fleches verticales** | Emissions | P(x_t \| s_t) gaussiennes conditionnelles |

**Propagation de messages** :
- **Forward** : P(s_t \| x_{1:t}) - information passee
- **Backward** : P(x_{t+1:T} \| s_t) - information future
- **Marginal** : P(s_t \| x_{1:T}) = Forward x Backward (normalise)

C'est cette structure en chaine qui permet au HMM de **lisser** les predictions en tenant compte des observations voisines.

## 4. Detection de Regimes Meteo

Cas d'usage classique : inferer une variable latente (le vrai temps) a partir d'observations bruitees (temperature).

| Regime | Temperature moyenne | Ecart-type |
|--------|---------------------|------------|
| Soleil | 22C | 2C |
| Pluie | 15C | 2C |

In [18]:
// Exemple : Detection de regimes meteo (Soleil/Pluie) a partir de temperature

// Soleil : temperature ~22C
// Pluie : temperature ~15C

double[] tempJour = { 21, 23, 22, 20, 15, 14, 16, 15, 14, 21, 22, 23 };
int Tmeteo = tempJour.Length;

Console.WriteLine("=== Detection Regimes Meteo ===");
Console.WriteLine($"\nTemperatures : {string.Join(", ", tempJour)}\n");

for (int t = 0; t < Tmeteo; t++)
{
    Variable<int> meteo = Variable.DiscreteUniform(2);
    Variable<double> temp = Variable.New<double>();
    
    using (Variable.Case(meteo, 0))  // Soleil
    {
        temp.SetTo(Variable.GaussianFromMeanAndVariance(22, 4));
    }
    using (Variable.Case(meteo, 1))  // Pluie
    {
        temp.SetTo(Variable.GaussianFromMeanAndVariance(15, 4));
    }
    
    temp.ObservedValue = tempJour[t];
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Discrete meteoPost = eng.Infer<Discrete>(meteo);
    string regime = meteoPost.GetProbs()[0] > 0.5 ? "Soleil" : "Pluie";
    
    Console.WriteLine($"Jour {t+1,2} : {tempJour[t]:F0}C -> {regime} (P={meteoPost.GetProbs().Max():F2})");
}

=== Detection Regimes Meteo ===

Temperatures : 21, 23, 22, 20, 15, 14, 16, 15, 14, 21, 22, 23

Compiling model...done.
Jour  1 : 21C -> Soleil (P=0,99)
Compiling model...done.
Jour  2 : 23C -> Soleil (P=1,00)
Compiling model...done.
Jour  3 : 22C -> Soleil (P=1,00)
Compiling model...done.
Jour  4 : 20C -> Soleil (P=0,93)
Compiling model...done.
Jour  5 : 15C -> Pluie (P=1,00)
Compiling model...done.
Jour  6 : 14C -> Pluie (P=1,00)
Compiling model...done.
Jour  7 : 16C -> Pluie (P=0,99)
Compiling model...done.
Jour  8 : 15C -> Pluie (P=1,00)
Compiling model...done.
Jour  9 : 14C -> Pluie (P=1,00)
Compiling model...done.
Jour 10 : 21C -> Soleil (P=0,99)
Compiling model...done.
Jour 11 : 22C -> Soleil (P=1,00)
Compiling model...done.
Jour 12 : 23C -> Soleil (P=1,00)


In [20]:
// Visualisation du graphe de facteurs pour la detection meteo
// Modele identique a la classification independante

Variable<int> meteoViz = Variable.DiscreteUniform(2).Named("meteo");
Variable<double> tempViz = Variable.New<double>().Named("temperature");

using (Variable.Case(meteoViz, 0))
{
    tempViz.SetTo(Variable.GaussianFromMeanAndVariance(22, 4).Named("emission_Soleil"));
}
using (Variable.Case(meteoViz, 1))
{
    tempViz.SetTo(Variable.GaussianFromMeanAndVariance(15, 4).Named("emission_Pluie"));
}

tempViz.ObservedValue = 18.0;

InferenceEngine engMeteoViz = new InferenceEngine();
engMeteoViz.Compiler.CompilerChoice = CompilerChoice.Roslyn;
engMeteoViz.ShowFactorGraph = true;

var _meteo = engMeteoViz.Infer(meteoViz);
Console.WriteLine("\nGraphe de facteurs genere pour le modele meteo.");

Compiling model...done.

Graphe de facteurs genere pour le modele meteo.


In [21]:
// Affichage du graphe meteo
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_28_20_62.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 2 
 
<!-- node1 -->
 
 node1 
 
 DiscreteUniform 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 size 
 
<!-- node2 -->
 
 node2 
 
 meteo 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node6 -->
 
 node6 
 
 18 
 
<!-- node2->node6 -->
 
 node2->node6 
 
 
 condition 
 
<!-- node3 -->
 
 node3 
 
 22 
 
<!-- node4 -->
 
 node4 
 
 GaussianFromMeanAndVariance 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 4 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 variance 
 
<!-- node7 -->
 
 node7 
 
 15 
 
<!-- node8 -->
 
 node8 
 
 GaussianFromMeanAndVariance 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 mean 
 
<!-- node8->node6 -->
 
 node8->node6 
 
 
 
<!-- node9 -->
 
 node9 
 
 4 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 variance


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Graphe de facteurs : Modele meteo (classification)

Structure identique a la detection d'anomalies - un modele de **melange gaussien** :

| Composante | Parametres |
|------------|------------|
| **Soleil** | N(22, 4) - temperature elevee |
| **Pluie** | N(15, 4) - temperature basse |

Le graphe montre le branchement conditionnel (Variable.Case) qui selectionne l'emission appropriee selon l'etat latent `meteo`.

> **Application** : Ce modele simple est equivalent a un classifieur de Bayes naive pour une seule feature (temperature).

**Resultats** : Detection correcte des deux regimes (Soleil jours 1-4 et 10-12, Pluie jours 5-9).

> **Point d'interet** : Le jour 4 (20C) a une confiance "seulement" 93% car la temperature est intermediaire entre les deux regimes.

## 5. Motif Finding (Bioinformatique)

### Probleme

Trouver des motifs conserves dans des sequences ADN.

### Modele

- Arriere-plan : nucleotides uniformes (A, C, G, T)
- Motif : positions avec distributions specifiques

Le probleme peut se formuler comme un HMM :

- **Etat Arriere-plan (B)** : $P(\text{base} | B) = \frac{1}{4}$ (uniforme)
- **Etat Motif (M)** : $P(\text{base} | M) \sim \text{Dir}(\alpha)$ (appris)

Un motif est significatif si le ratio de vraisemblance $\frac{P(\text{motif} | M)}{P(\text{motif} | B)} > 1$.

In [22]:
// Motif Finding simplifie

// Sequences ADN (codees : A=0, C=1, G=2, T=3)
int[][] sequences = {
    new[] { 0, 1, 2, 0, 0, 1, 3, 2, 0, 1 },  // ...ACGAACTGAC
    new[] { 3, 0, 0, 1, 2, 3, 0, 1, 2, 0 },  // ...TAACGTACGA
    new[] { 2, 0, 0, 1, 1, 0, 3, 0, 1, 2 }   // ...GAACCATACG
};

// Motif cible : "AAC" aux positions 2-4 dans seq 1, 1-3 dans seq 2, 1-3 dans seq 3

Console.WriteLine("=== Motif Finding ===");
Console.WriteLine("\nSequences ADN (A=0, C=1, G=2, T=3) :");

string[] bases = { "A", "C", "G", "T" };
for (int s = 0; s < sequences.Length; s++)
{
    string seqStr = string.Join("", sequences[s].Select(n => bases[n]));
    Console.WriteLine($"  Seq {s+1} : {seqStr}");
}

Console.WriteLine("\nRecherche du motif conserve...");

=== Motif Finding ===

Sequences ADN (A=0, C=1, G=2, T=3) :
  Seq 1 : ACGAACTGAC
  Seq 2 : TAACGTACGA
  Seq 3 : GAACCATACG

Recherche du motif conserve...


L'approche la plus simple est le **comptage de k-mers** : extraire tous les k-mers, compter leurs occurrences, et identifier les sur-representes.

In [23]:
// Comptage des k-mers
int motifLen = 3;
var kmerCounts = new Dictionary<string, int>();

foreach (var seq in sequences)
{
    for (int i = 0; i <= seq.Length - motifLen; i++)
    {
        string kmer = string.Join("", seq.Skip(i).Take(motifLen).Select(n => bases[n]));
        if (!kmerCounts.ContainsKey(kmer)) kmerCounts[kmer] = 0;
        kmerCounts[kmer]++;
    }
}

var topKmers = kmerCounts.OrderByDescending(kv => kv.Value).Take(5);

Console.WriteLine("\nTop 5 k-mers (longueur 3) :");
foreach (var kv in topKmers)
{
    Console.WriteLine($"  {kv.Key} : {kv.Value} occurrences");
}

Console.WriteLine($"\n=> Motif candidat : {topKmers.First().Key}");


Top 5 k-mers (longueur 3) :
  ACG : 4 occurrences
  AAC : 3 occurrences
  CGA : 2 occurrences
  GAA : 2 occurrences
  TAC : 2 occurrences

=> Motif candidat : ACG


**Resultats** : "ACG" (4 occurrences) et "AAC" (3 occurrences, une par sequence) sont les motifs candidats.

> **Note biologique** : "AAC" est particulierement interessant car present exactement une fois par sequence, suggerant une conservation fonctionnelle.

## 6. Exercice : Detection d'Anomalies

### Enonce

Utilisez un HMM pour detecter des periodes anormales dans une serie temporelle de ventes.

Detection d'anomalies dans les series temporelles de ventes :

| Scenario | Caracteristiques | Action |
|----------|------------------|--------|
| Promotion | Hausse temporaire | Optimiser le stock |
| Rupture stock | Baisse soudaine | Reapprovisionner |
| Tendance | Evolution graduelle | Ajuster previsions |

In [24]:
// EXERCICE : Detection d'anomalies dans les ventes

// Ventes journalieres (normal ~100, promo ~200)
double[] ventes = { 98, 105, 102, 99, 195, 210, 205, 198, 103, 97, 101, 100 };

Console.WriteLine("=== Detection Periodes de Promotion ===");
Console.WriteLine($"\nVentes : {string.Join(", ", ventes.Select(v => v.ToString("F0")))}\n");

for (int t = 0; t < ventes.Length; t++)
{
    Variable<int> regime = Variable.DiscreteUniform(2);
    Variable<double> venteVar = Variable.New<double>();
    
    using (Variable.Case(regime, 0))  // Normal
    {
        venteVar.SetTo(Variable.GaussianFromMeanAndVariance(100, 100));
    }
    using (Variable.Case(regime, 1))  // Promotion
    {
        venteVar.SetTo(Variable.GaussianFromMeanAndVariance(200, 100));
    }
    
    venteVar.ObservedValue = ventes[t];
    
    InferenceEngine eng = new InferenceEngine();
    eng.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Discrete regimePost = eng.Infer<Discrete>(regime);
    string etat = regimePost.GetProbs()[1] > 0.5 ? "PROMO" : "Normal";
    
    Console.WriteLine($"Jour {t+1,2} : {ventes[t],3:F0} -> {etat} (P={regimePost.GetProbs().Max():F2})");
}

Console.WriteLine("\n=> Jours 5-8 detectes comme periode de promotion");

=== Detection Periodes de Promotion ===

Ventes : 98, 105, 102, 99, 195, 210, 205, 198, 103, 97, 101, 100

Compiling model...done.
Jour  1 :  98 -> Normal (P=1,00)
Compiling model...done.
Jour  2 : 105 -> Normal (P=1,00)
Compiling model...done.
Jour  3 : 102 -> Normal (P=1,00)
Compiling model...done.
Jour  4 :  99 -> Normal (P=1,00)
Compiling model...done.
Jour  5 : 195 -> PROMO (P=1,00)
Compiling model...done.
Jour  6 : 210 -> PROMO (P=1,00)
Compiling model...done.
Jour  7 : 205 -> PROMO (P=1,00)
Compiling model...done.
Jour  8 : 198 -> PROMO (P=1,00)
Compiling model...done.
Jour  9 : 103 -> Normal (P=1,00)
Compiling model...done.
Jour 10 :  97 -> Normal (P=1,00)
Compiling model...done.
Jour 11 : 101 -> Normal (P=1,00)
Compiling model...done.
Jour 12 : 100 -> Normal (P=1,00)

=> Jours 5-8 detectes comme periode de promotion


In [25]:
// Visualisation du graphe pour la detection de promotions
Variable<int> regimeViz = Variable.DiscreteUniform(2).Named("regime_ventes");
Variable<double> venteViz = Variable.New<double>().Named("ventes_journalieres");

using (Variable.Case(regimeViz, 0))
{
    venteViz.SetTo(Variable.GaussianFromMeanAndVariance(100, 100).Named("emission_Normal"));
}
using (Variable.Case(regimeViz, 1))
{
    venteViz.SetTo(Variable.GaussianFromMeanAndVariance(200, 100).Named("emission_Promo"));
}

venteViz.ObservedValue = 150.0;

InferenceEngine engVenteViz = new InferenceEngine();
engVenteViz.Compiler.CompilerChoice = CompilerChoice.Roslyn;
engVenteViz.ShowFactorGraph = true;

var _regime = engVenteViz.Infer(regimeViz);
Console.WriteLine("\nGraphe de facteurs genere pour la detection de promotions.");

Compiling model...done.

Graphe de facteurs genere pour la detection de promotions.


In [26]:
// Affichage du graphe de detection de promotions
display(HTML(FactorGraphHelper.GetLatestFactorGraphHtml()));

Model_01_31_26_19_28_22_80.svg 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.2 (20260124.0452)
 -->
<!-- Title: Model Pages: 1 -->
 
 
 Model 
 
<!-- node0 -->
 
 node0 
 
 2 
 
<!-- node1 -->
 
 node1 
 
 DiscreteUniform 
 
<!-- node0->node1 -->
 
 node0->node1 
 
 
 size 
 
<!-- node2 -->
 
 node2 
 
 regime_ventes 
 
<!-- node1->node2 -->
 
 node1->node2 
 
 
 
<!-- node6 -->
 
 node6 
 
 150 
 
<!-- node2->node6 -->
 
 node2->node6 
 
 
 condition 
 
<!-- node3 -->
 
 node3 
 
 100 
 
<!-- node4 -->
 
 node4 
 
 GaussianFromMeanAndVariance 
 
<!-- node3->node4 -->
 
 node3->node4 
 
 
 mean 
 
<!-- node4->node6 -->
 
 node4->node6 
 
 
 
<!-- node5 -->
 
 node5 
 
 100 
 
<!-- node5->node4 -->
 
 node5->node4 
 
 
 variance 
 
<!-- node7 -->
 
 node7 
 
 200 
 
<!-- node8 -->
 
 node8 
 
 GaussianFromMeanAndVariance 
 
<!-- node7->node8 -->
 
 node7->node8 
 
 
 mean 
 
<!-- node8->node6 -->
 
 node8->node6 
 
 
 
<!-- node9 -->
 
 node9 
 
 100 
 
<!-- node9->node8 -->
 
 node9->node8 
 
 
 variance


warning CS1701: En supposant que la référence d'assembly 'Microsoft.AspNetCore.Html.Abstractions, Version=2.3.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' utilisée par 'Microsoft.DotNet.Interactive' correspond à l'identité 'Microsoft.AspNetCore.Html.Abstractions, Version=10.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' de 'Microsoft.AspNetCore.Html.Abstractions', il se peut que vous deviez fournir une stratégie runtime



### Graphe de facteurs : Detection de promotions

Meme structure que les modeles precedents - classification par melange gaussien :

| Regime | Distribution | Interpretation |
|--------|--------------|----------------|
| **Normal** | N(100, 100) | Ventes moyennes ~100 unites |
| **Promo** | N(200, 100) | Ventes elevees ~200 unites |

**Extension possible** : Pour une detection plus robuste des periodes promotionnelles, on ajouterait des **transitions Markoviennes** comme dans la section 3.3. Cela permettrait de :

1. Penaliser les changements frequents Normal <-> Promo
2. Detecter des periodes coherentes (promotion sur plusieurs jours consecutifs)
3. Lisser les observations ambigues (ex: 150 unites) selon le contexte

**Resultats** : Detection parfaite - jours 5-8 identifies comme periode promotionnelle (ventes ~200 vs ~100 en normal).

## 7. Resume

| Concept | Description |
|---------|-------------|
| **HMM** | Modele a etats caches avec dependances temporelles |
| **Emissions** | Distribution des observations selon l'etat |
| **Transitions** | Probabilites de changement d'etat |
| **Viterbi** | Algorithme pour trouver la sequence d'etats optimale |
| **Forward-Backward** | Calcul des probabilites marginales |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|--------------|
| Comprendre Variable.Switch | [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb) |
| Comparer EP vs VMP pour HMM | [Infer-13-Debugging](Infer-13-Debugging.ipynb) Section 4 |
| Debugger des problemes de convergence | [Infer-13-Debugging](Infer-13-Debugging.ipynb) |
| Trouver une definition (HMM, Viterbi, etc.) | [Glossaire](Infer-Glossary.md) |

---

## Prochaine etape

Dans [Infer-12-Recommenders](Infer-12-Recommenders.ipynb), nous explorerons :

- Les systemes de recommandation
- La factorisation matricielle
- Le modele ClickModel pour sources multiples